# Install dependencies

In [6]:
!pip install pandas ipython-sql


In [7]:
!pip install beautifulsoup4 requests lxml

# Add imports

In [8]:
import pandas as pd

In [9]:
from bs4 import BeautifulSoup
import requests

# Create Datasets for all pharmacies in London separated by borough

## Download pharmacy list from https://opendata.nhsbsa.net/dataset/consolidated-pharmaceutical-list

In [13]:
# Consolidated Pharmaceutical List: 2023-24 Q4
# https://opendata.nhsbsa.net/dataset/240d142d-df82-4e97-b051-12371519e4e1/resource/d36c355a-631d-4ddb-bc34-5a0315565e12/download/consol_pharmacy_list_202324q4.csv

CONSOLIDATED_PHARMACEUTICAL_LIST_CSV = "https://opendata.nhsbsa.net/dataset/240d142d-df82-4e97-b051-12371519e4e1/resource/d36c355a-631d-4ddb-bc34-5a0315565e12/download/consol_pharmacy_list_202324q4.csv"
pharmacy_df = pd.read_csv(CONSOLIDATED_PHARMACEUTICAL_LIST_CSV)

## Generate London Pharmacies Dataframe

In [17]:
lon_health_wellbeing_boards = ["CITY OF LONDON", "KENSINGTON AND CHELSEA", "BRENT", "HAMMERSMITH AND FULHAM", "HOUNSLOW", "ISLINGTON", "SUTTON", "WALTHAM FOREST", "KINGSTON", "BARNET", "SOUTHWARK", "CAMDEN", "HILLINGDON", "WESTMINSTER", "CROYDON", "HARINGEY", "GREENWICH", "LEWISHAM", "BARKING AND DAGENHAM", "MERTON", "LAMBETH", "EALING", "BEXLEY", "HARROW", "REDBRIDGE", "HAVERING", "HACKNEY", "TOWER HAMLETS"]

In [18]:
lon_pharmacy_df = pharmacy_df[["HEALTH_AND_WELLBEING_BOARD", "PHARMACY_TRADING_NAME", "ADDRESS_FIELD1", "ADDRESS_FIELD2", "ADDRESS_FIELD3", "ADDRESS_FIELD4", "POST_CODE"]]

In [19]:
lon_pharmacy_df.head()

,HEALTH_AND_WELLBEING_BOARD,PHARMACY_TRADING_NAME,ADDRESS_FIELD1,ADDRESS_FIELD2,ADDRESS_FIELD3,ADDRESS_FIELD4,POST_CODE
0,STOCKPORT,ROWLANDS PHARMACY,61 ARUNDEL AVENUE,HAZEL GROVE,STOCKPORT,,SK7 5LD
1,TOWER HAMLETS,MEDICHEM,100 WHITEHORSE LANE,STEPNEY,LONDON,,E1 4LR
2,KENT,ALLCURES PLC,HEADCORN SURGERY,"GRIGG LANE, HEADCORN",ASHFORD,KENT,TN27 9AA
3,NOTTINGHAMSHIRE,GALEXA PHARMACY,61 ANNESLEY ROAD,HUCKNALL,NOTTINGHAM,NOTTINGHAMSHIRE,NG15 7DR
4,WARWICKSHIRE,GRENDON PHARMACY,150 BOOT HILL,,BADDESLEY ENSOR,WARWICKSHIRE,CV9 2EW


In [20]:
lon_pharmacy_df = lon_pharmacy_df.loc[pharmacy_df.HEALTH_AND_WELLBEING_BOARD.isin(lon_health_wellbeing_boards)]

In [22]:
lon_pharmacy_df.head()

,HEALTH_AND_WELLBEING_BOARD,PHARMACY_TRADING_NAME,ADDRESS_FIELD1,ADDRESS_FIELD2,ADDRESS_FIELD3,ADDRESS_FIELD4,POST_CODE
1,TOWER HAMLETS,MEDICHEM,100 WHITEHORSE LANE,STEPNEY,LONDON,,E1 4LR
11,HACKNEY,FA STRANGE CHEMIST,185 LOWER CLAPTON ROAD,NaN,LONDON,NaN,E5 8EQ
12,HAVERING,TESCO INSTORE PHARMACY,TESCO SUPERSTORE,BRIDGE ROAD,RAINHAM,ESSEX,RM13 9YZ
16,REDBRIDGE,WELL,2 JUBILEE PARADE,SNAKES LANE,WOODFORD GREEN,ESSEX,IG8 7QF
20,HARROW,FAIRVIEW PHARMACY,293-5 BURNT OAK BROADWAY,,EDGWARE,MIDDLESEX,HA8 5ED


## Create Dataframe for each London borough from London Pharmacies Dataframe

In [26]:
def create_borough_df(borough_name):
    return lon_pharmacy_df.loc[lon_pharmacy_df.HEALTH_AND_WELLBEING_BOARD.isin([borough_name])]

In [34]:
# Create df for each London health and wellbeing board

barnet_pharmacy_df = create_borough_df("BARNET")
barking_and_dagenham_pharmacy_df = create_borough_df("BARKING AND DAGENHAM")
bexley_pharmacy_df = create_borough_df("BEXLEY")
brent_pharmacy_df = create_borough_df("BRENT")

camden_pharmacy_df = create_borough_df("CAMDEN")
city_of_london_pharmacy_df = create_borough_df("CITY OF LONDON")
croydon_pharmacy_df = create_borough_df("CROYDON")

ealing_pharmacy_df = create_borough_df("EALING")

greenwich_pharmacy_df = create_borough_df("GREENWICH")

hackney_pharmacy_df = create_borough_df("HACKNEY")
haringey_pharmacy_df = create_borough_df("HARINGEY")
hammersmith_and_fulham_pharmacy_df = create_borough_df("HAMMERSMITH AND FULHAM")
harrow_pharmacy_df = create_borough_df("HARROW")
havering_pharmacy_df = create_borough_df("HAVERING")
hillingdon_pharmacy_df = create_borough_df("HILLINGDON")
hounslow_pharmacy_df = create_borough_df("HOUNSLOW")

islington_pharmacy_df = create_borough_df("ISLINGTON")

kensington_and_chelsea_pharmacy_df = create_borough_df("KENSINGTON AND CHELSEA")
kingston_pharmacy_df = create_borough_df("KINGSTON")

lambeth_pharmacy_df = create_borough_df("LAMBETH")
lewisham_pharmacy_df = create_borough_df("LEWISHAM")

merton_pharmacy_df = create_borough_df("MERTON")

redbridge_pharmacy_df = create_borough_df("REDBRIDGE")

southwark_pharmacy_df = create_borough_df("SOUTHWARK")
sutton_pharmacy_df = create_borough_df("SUTTON")

tower_hamlets_pharmacy_df = create_borough_df("TOWER HAMLETS")

waltham_forest_pharmacy_df = create_borough_df("WALTHAM FOREST")
westminster_pharmacy_df = create_borough_df("WESTMINSTER")

# Scrape phone numbers from Google and add to data set

In [12]:
headers = {
  "User-Agent":
  "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

params = {
  "hl": "en",
  "gl": "uk" # contry to search from. uk = United Kingdom. us = United States
}

query = "university of glasgow"
query = f"https://www.google.com/search?q={query} phone"

response = requests.get(query, headers=headers, params=params)
response.encoding='utf-8'

In [56]:
for name in borough_dataframes:
    df_var_name = retrieve_name(name)
    csv_name = df_var_name.replace('_pharmacy_df', '')
    name.to_csv('./' + csv_name + '_pharmacies.csv')

In [13]:
page = response.text
soup = BeautifulSoup(page, 'lxml')

try:
    phone = soup.select_one(".pwimAb").text
except: phone = "not found"
print(phone)

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

# Create CSV for each borough

In [50]:
import inspect

def retrieve_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    names = [var_name for var_name, var_val in callers_local_vars if var_val is var]
    if len(names) > 0:
            return names[0]

In [51]:
borough_dataframes = [
    barnet_pharmacy_df,
    barking_and_dagenham_pharmacy_df,
    bexley_pharmacy_df,
    brent_pharmacy_df,
    camden_pharmacy_df,
    city_of_london_pharmacy_df,
    croydon_pharmacy_df,
    ealing_pharmacy_df,
    greenwich_pharmacy_df,
    hackney_pharmacy_df,
    haringey_pharmacy_df,
    hammersmith_and_fulham_pharmacy_df,
    harrow_pharmacy_df,
    havering_pharmacy_df,
    hillingdon_pharmacy_df,
    hounslow_pharmacy_df,
    islington_pharmacy_df,
    kensington_and_chelsea_pharmacy_df,
    kingston_pharmacy_df,
    lambeth_pharmacy_df,
    lewisham_pharmacy_df,
    merton_pharmacy_df,
    redbridge_pharmacy_df,
    southwark_pharmacy_df,
    sutton_pharmacy_df,
    tower_hamlets_pharmacy_df,
    waltham_forest_pharmacy_df,
    westminster_pharmacy_df
]